### Imports 

In [1]:
# Imports
import torch as pt
from torch import nn

print(f"Torch version: {pt.__version__}")

# if pt.cuda.is_available():
#     device = 'cuda'
# if pt.backends.mps.is_available():
#     device = 'mps'
# else:
#     device= 'cpu'
device = 'cpu'
print(f'device: {device}')

Torch version: 2.0.1
device: cpu


### Downloading a custom dataset

In [2]:
from pathlib import Path
import importLib
from sys import path
import zipfile


# Create directory
data_path = Path(f"{path[0]}/data")
image_path = data_path / 'pizza_steak_sushi'
if image_path.exists():
    print('Already exists')
else:
    image_path.mkdir(parents=True)


# Download pizza, steak and sushi data
# open skapar en zip fil som sedan fylls genom request
importLib.import_from_github('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip',directory=data_path)
with zipfile.ZipFile(data_path/'pizza_steak_sushi.zip', 'r') as zip_ref:
    print('Unzipping pizza, steak and sushi data')
    zip_ref.extractall(image_path)
Path.unlink(data_path/'pizza_steak_sushi.zip')


Already exists
/Users/gustavgamstedt/Desktop/github to hemma/PyTorch/04/data/pizza_steak_sushi.zip doesn't exist, download
Unzipping pizza, steak and sushi data


In [3]:
# Setup training and testing paths
train_dir = image_path / 'train'
test_dir = image_path / 'test'

train_dir, test_dir

(PosixPath('/Users/gustavgamstedt/Desktop/github to hemma/PyTorch/04/data/pizza_steak_sushi/train'),
 PosixPath('/Users/gustavgamstedt/Desktop/github to hemma/PyTorch/04/data/pizza_steak_sushi/test'))

### Create dataset and dataloaders

In [4]:
from torchvision import transforms
simple_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

In [5]:
from torch.utils.data import DataLoader
from torchvision import datasets

train_dataset = datasets.ImageFolder(root = train_dir, transform=simple_transform)
test_dataset = datasets.ImageFolder(root = test_dir, transform=simple_transform)

In [6]:
import os
BATCH_SIZE = 32
NUM_WORKERS = round(os.cpu_count()*(3/4))
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    num_workers=NUM_WORKERS,
    shuffle=False
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=32,
    num_workers=NUM_WORKERS,
    shuffle=False
)

### Create model

In [7]:
from torch import nn

class ModelWithoutAugmentation(nn.Module):
    def __init__(self, input_features:int,output_features:int, hidden_units:int=10):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(input_features, hidden_units,
                      kernel_size=3, stride=1, padding=0),
            nn.ReLU(),


            nn.Conv2d(hidden_units, hidden_units,
                      kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units,
                      kernel_size=3, stride=1, padding=0),
            nn.ReLU(),


            nn.Conv2d(hidden_units, hidden_units,
                      kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units*13*13, output_features)
        )
    def forward(self, X:pt.Tensor) -> pt.Tensor:
        X_change = self.conv_block_1(X)
        X_change = self.conv_block_2(X_change)
        # print(X_change.shape)
        X_change = self.classifier(X_change)
        return X_change

In [8]:
pt.manual_seed(42)
model0 = ModelWithoutAugmentation(input_features=3, output_features=len(train_dataset.classes), hidden_units=10).to(device)
model0

ModelWithoutAugmentation(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

#### Testing model with random data

In [9]:
imgs, labels = next(iter(train_dataloader))
print(imgs.shape, len(labels))
# model0(imgs[0].unsqueeze(0))
model0(imgs.to(device))

torch.Size([32, 3, 64, 64]) 32


tensor([[ 1.7012e-02, -5.3149e-03,  1.2772e-02],
        [ 1.8558e-02, -1.8290e-03,  9.5356e-03],
        [ 2.1909e-02, -4.9545e-03,  9.4219e-03],
        [ 2.3536e-02, -5.1556e-03,  1.0510e-02],
        [ 2.1787e-02,  6.3863e-05,  7.6012e-03],
        [ 2.0530e-02, -1.1189e-03,  1.1084e-02],
        [ 2.1927e-02, -1.7779e-03,  8.1963e-03],
        [ 2.0188e-02, -2.4430e-03,  1.0628e-02],
        [ 2.2015e-02, -1.4427e-03,  7.8839e-03],
        [ 2.1256e-02, -1.6102e-03,  9.5531e-03],
        [ 1.9500e-02, -2.4019e-03,  8.3024e-03],
        [ 2.1457e-02, -1.8471e-03,  7.7899e-03],
        [ 2.0759e-02,  1.9331e-04,  7.7705e-03],
        [ 2.1412e-02, -1.9498e-03,  1.2461e-02],
        [ 1.8300e-02,  1.5043e-03,  1.0491e-02],
        [ 2.2685e-02, -2.1673e-03,  9.2847e-03],
        [ 2.3170e-02, -1.5725e-03,  8.5609e-03],
        [ 2.0840e-02, -1.9240e-03,  9.4237e-03],
        [ 2.2364e-02, -1.1584e-03,  7.3213e-03],
        [ 2.3398e-02, -3.6900e-03,  1.0476e-02],
        [ 2.4240e-02

### Summarize a model

In [10]:
try:
    import torchinfo
except ModuleNotFoundError:
    print('Module not found, installing module')
    !pip3 install torchinfo

In [11]:
torchinfo.summary(model0, input_size=[32,3,64,64],device=device)

Layer (type:depth-idx)                   Output Shape              Param #
ModelWithoutAugmentation                 [32, 3]                   --
├─Sequential: 1-1                        [32, 10, 30, 30]          --
│    └─Conv2d: 2-1                       [32, 10, 62, 62]          280
│    └─ReLU: 2-2                         [32, 10, 62, 62]          --
│    └─Conv2d: 2-3                       [32, 10, 60, 60]          910
│    └─ReLU: 2-4                         [32, 10, 60, 60]          --
│    └─MaxPool2d: 2-5                    [32, 10, 30, 30]          --
├─Sequential: 1-2                        [32, 10, 13, 13]          --
│    └─Conv2d: 2-6                       [32, 10, 28, 28]          910
│    └─ReLU: 2-7                         [32, 10, 28, 28]          --
│    └─Conv2d: 2-8                       [32, 10, 26, 26]          910
│    └─ReLU: 2-9                         [32, 10, 26, 26]          --
│    └─MaxPool2d: 2-10                   [32, 10, 13, 13]          --
├─Sequentia

### Create train and test loop functions

In [12]:
from torch.utils.data import DataLoader
def train_step(model: pt.nn.Module, 
               dataloader:DataLoader, 
               loss_fn: pt.nn.Module, 
               optimizer:pt.optim.Optimizer, 
               device:pt.device, 
               show:bool=False):
    """Performs a training step with model trying to learn on data_loader

    args:
        model: the model which will be trained on
        dataloader: A generator like loader for the data
        optimizer: Optimizer which optimizes the code through gradient descend
        loss_fn: function which calculates how far from the right answer each of the predictions were
        accuracy_fn: function which calculates how meny predictions were right
        device: chosen device for the neural network to run on (cpu/gpu/tpu)
        show: if true display the loss and acc in console 
        
    returns:
        (loss, accuracy)"""
    # Put model in training mode
    model.train()
     
    # Setup train loss and train accuracy values
    train_loss, train_acc = 0,0

    # Loop through data loader batches
    for X,y in dataloader:
        # Send data to target device
        X,y = X.to(device), y.to(device)

        y_logits = model(X)
        
        loss = loss_fn(y_logits, y)
        train_loss+=loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_preds = pt.argmax(pt.softmax(y_logits, dim=1), dim=1) # Softmax is actually unnecessary, but can be useful for visualization and also to give completeness
        train_acc += pt.eq(y_preds, y).sum().item()/len(y_preds)
    train_loss /= len(dataloader)
    train_acc  /= len(dataloader)

    if show:
        print(f'Train loss: {train_loss} | Train acc: {train_acc}')
    return train_loss, train_acc

In [13]:
def test_step(model: pt.nn.Module, 
              dataloader:DataLoader, 
              loss_fn: pt.nn.Module, 
              device:pt.device, 
              show:bool=False):
    """Performs a testing loop step on model going over data_loader.

    args:
        model: the model which will be trained on
        dataloader: A generator like loader for the data
        loss_fn: function which calculates how far from the right answer each of the predictions were
        accuracy_fn: function which calculates how meny predictions were right
        device: chosen device for the neural network to run on (cpu/gpu/tpu)
        show: if true display the loss and acc in console 

    returns:
        (loss, accuracy)"""
    test_acc, test_loss = 0,0
    
    model.eval()
    with pt.inference_mode():
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)
            y_logits = model(X)
            loss = loss_fn(y_logits, y)
            test_loss+=loss.item()

            y_preds = pt.argmax(pt.softmax(y_logits, dim=1), dim=1)
            test_acc += pt.eq(y_preds, y).sum().item()/len(y_preds)
    test_loss /= len(dataloader)
    test_acc  /= len(dataloader)
    if show:
        print(f'Test loss: {test_loss} | Test acc: {test_acc}')

In [14]:
loss_fn = nn.CrossEntropyLoss()

optimizer = pt.optim.Adam(model0.parameters(), lr=0.001)

In [15]:
print(device)


cpu


In [16]:
from ml_funcs import Model_operations, Timer
from tqdm.notebook import tqdm
epochs = 5
timer = Timer()

for epoch in range(epochs):
    print(f'Epoch: {epoch}')
    Model_operations.train_step(
        model=model0,
        dataloader=train_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        device=device,
        show=True)
    Model_operations.test_step(model0, test_dataloader, loss_fn, device, True)
    timer.interval()

timer.show_as_print()

Epoch: 0
Train loss: 1.214791752398014 | Train acc: 0.3046875
Test loss: 1.1588222185770671 | Test acc: 0.2604166666666667
Epoch: 1
Train loss: 1.105965219438076 | Train acc: 0.3046875
Test loss: 1.1224364042282104 | Test acc: 0.2604166666666667
Epoch: 2
Train loss: 1.0975951552391052 | Train acc: 0.3046875
Test loss: 1.1026708682378132 | Test acc: 0.2604166666666667
Epoch: 3
Train loss: 1.0952743589878082 | Train acc: 0.3046875
Test loss: 1.0911263624827068 | Test acc: 0.2604166666666667
Epoch: 4
Train loss: 1.0944082736968994 | Train acc: 0.53125
Test loss: 1.0834309260050456 | Test acc: 0.5416666666666666
Timer: by GGisMee
Interval time: 
Interval 1: 36.29
Interval 2: 36.27
Interval 3: 35.83
Interval 4: 35.89
Interval 5: 37.47



In [17]:
print(Model_operations.eval_model(model0, train_dataloader, loss_fn, device))

{'model_name': 'ModelWithoutAugmentation', 'model_loss': 1.0938726663589478, 'model_acc': 0.40234375}
